In [1]:
import pandas as pd
import numpy as np
import os
import sys
import json

In [9]:
path = "/home/al2644/research/lib/SkyThought/deepseek-ai_DeepSeek-R1-Distill-Qwen-1.5B_aime24_76987d/summary.json"
with open(path, 'rb') as f:
    results = json.load(f)

In [10]:
results

{'configuration': {'task': {'name': 'aime24',
   'config': {'handler': 'aime',
    'dataset_path': 'AI-MO/aimo-validation-aime',
    'dataset_subset': None,
    'dataset_split': 'train',
    'dataset_kwargs': {},
    'question_key': 'problem',
    'answer_key': 'answer',
    'templating_parameters': {'template': 'Return your final response within \\boxed{{}}. {prompt}'},
    'preprocess_config': {'url': '2024'}},
   'start': 0,
   'end': -1},
  'model': {'name': 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B',
   'config': {'model_id': 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B',
    'name': 'DeepSeek-R1-Distill-Qwen-1.5B',
    'system_prompt': None,
    'user_template': None,
    'assistant_prefill': None}},
  'backend': {'name': 'vllm', 'backend_args': {}},
  'sampling_params': {'temperature': 0, 'top_p': 1, 'max_tokens': 32768}},
 'total_completion_tokens': 750392,
 'avg_completion_tokens': 25013.067,
 'total_prompt_tokens': 3631,
 'avg_prompt_tokens': 121.033,
 'accuracy': 0.2333,
 'pa

In [2]:
from datasets import load_dataset

In [17]:
train_dataset = load_dataset("SynthLabsAI/Big-Math-RL-Verified")['train']

In [43]:
math500 = load_dataset("HuggingFaceH4/MATH-500")['test']
math500 = pd.DataFrame(math500)

In [19]:
dataset_df = pd.DataFrame(train_dataset)

In [29]:
aime_24 = load_dataset("Maxwell-Jia/AIME_2024")['train']

In [33]:
answer_str = [str(ans) for ans in aime_24['Answer']]

In [36]:
subset = dataset_df[dataset_df['source'] == 'amc_aime']

In [51]:
math_df = dataset_df[dataset_df['source'] == 'math']

In [60]:
dataset_df['problem'][0]

'Given $p$: $|4x-3|\\leqslant 1$ and $q$: $x^{2}-(2a+1)x+a^{2}+a\\leqslant 0$, find the range of values for $a$ if $p$ is a necessary but not sufficient condition for $q$.'

# TRAIN Dataset

In [51]:
open_r1_ds = load_dataset("open-r1/OpenR1-Math-220k")

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

In [52]:
open_r1_ds

DatasetDict({
    train: Dataset({
        features: ['problem', 'solution', 'answer', 'problem_type', 'question_type', 'source', 'uuid', 'is_reasoning_complete', 'generations', 'correctness_math_verify', 'correctness_llama', 'finish_reasons', 'correctness_count', 'messages'],
        num_rows: 93733
    })
})

In [64]:
idx = 1301
data = open_r1_ds['train'][idx]
print(data['problem'])
print("=" * 20)
print(data['answer'])
print("=" * 20)
print(data['generations'][0])

2. In a computer game, a turtle moves across a grid on the computer screen, which contains 11 columns and 5 rows. Initially, it is located at the bottom-left corner of the screen - on the cell with coordinates $(0,0)$. If the program instructs the turtle to move off the screen, it reappears on the opposite side - for example, taking one step up from cell $(3,4)$, the turtle will end up in cell $(3,0)$. Where will the turtle be after executing the following program:
1) 1 step down; 2) 2 steps to the right; 3) 3 steps up; 4) 4 steps to the left; 5) 5 steps down; 6) 6 steps to the right; .. ; 2016) 2016 steps to the left; 2017) 2017 steps down?
(4,1)
<think>
Okay, so I need to figure out where the turtle ends up after executing a long series of movements on this 11x5 grid. The turtle starts at (0,0), which is the bottom-left corner. The movements are in a pattern: first 1 step down, then 2 steps right, 3 steps up, 4 steps left, 5 steps down, 6 steps right, and so on, alternating direction

In [54]:
data.keys()

dict_keys(['problem', 'solution', 'answer', 'problem_type', 'question_type', 'source', 'uuid', 'is_reasoning_complete', 'generations', 'correctness_math_verify', 'correctness_llama', 'finish_reasons', 'correctness_count', 'messages'])

# Tokenizer

In [1]:
from transformers import AutoTokenizer

In [2]:
base_tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B")
r1_tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

In [4]:
r1_tokenizer

LlamaTokenizerFast(name_or_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', vocab_size=151643, model_max_length=16384, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<｜begin▁of▁sentence｜>', 'eos_token': '<｜end▁of▁sentence｜>', 'pad_token': '<｜end▁of▁sentence｜>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<｜end▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<｜User｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	151645: AddedToken("<｜Assistant｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	151646: AddedToken("<｜begin▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151647: AddedToken("<|EOT|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	151648: AddedToken("<think>", rstrip=False

In [3]:
base_tokenizer

Qwen2TokenizerFast(name_or_path='Qwen/Qwen2.5-1.5B', vocab_size=151643, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|endoftext|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, single_word=False, norma

In [65]:
DEFAULT_CHAT_TEMPLATE = """{% if not add_generation_prompt is defined %}{% set add_generation_prompt = false %}{% endif %}{% set ns = namespace(is_first=false, is_tool=false, is_output_first=true, system_prompt='') %}{%- for message in messages %}{%- if message['role'] == 'system' %}{% set ns.system_prompt = message['content'] %}{%- endif %}{%- endfor %}{{bos_token}}{{ns.system_prompt}}{%- for message in messages %}{%- if message['role'] == 'user' %}{%- set ns.is_tool = false -%}{{'<｜User｜>' + message['content']}}{%- endif %}{%- if message['role'] == 'assistant' and message['content'] is none %}{%- set ns.is_tool = false -%}{%- for tool in message['tool_calls']%}{%- if not ns.is_first %}{{'<｜Assistant｜><｜tool▁calls▁begin｜><｜tool▁call▁begin｜>' + tool['type'] + '<｜tool▁sep｜>' + tool['function']['name'] + '\\n' + '```json' + '\\n' + tool['function']['arguments'] + '\\n' + '```' + '<｜tool▁call▁end｜>'}}{%- set ns.is_first = true -%}{%- else %}{{'\\n' + '<｜tool▁call▁begin｜>' + tool['type'] + '<｜tool▁sep｜>' + tool['function']['name'] + '\\n' + '```json' + '\\n' + tool['function']['arguments'] + '\\n' + '```' + '<｜tool▁call▁end｜>'}}{{'<｜tool▁calls▁end｜><｜end▁of▁sentence｜>'}}{%- endif %}{%- endfor %}{%- endif %}{%- if message['role'] == 'assistant' and message['content'] is not none %}{%- if ns.is_tool %}{{'<｜tool▁outputs▁end｜>' + message['content'] + '<｜end▁of▁sentence｜>'}}{%- set ns.is_tool = false -%}{%- else %}{% set content = message['content'] %}{{'<｜Assistant｜>' + content + '<｜end▁of▁sentence｜>'}}{%- endif %}{%- endif %}{%- if message['role'] == 'tool' %}{%- set ns.is_tool = true -%}{%- if ns.is_output_first %}{{'<｜tool▁outputs▁begin｜><｜tool▁output▁begin｜>' + message['content'] + '<｜tool▁output▁end｜>'}}{%- set ns.is_output_first = false %}{%- else %}{{'\\n<｜tool▁output▁begin｜>' + message['content'] + '<｜tool▁output▁end｜>'}}{%- endif %}{%- endif %}{%- endfor -%}{% if ns.is_tool %}{{'<｜tool▁outputs▁end｜>'}}{% endif %}{% if add_generation_prompt and not ns.is_tool %}{{'<｜Assistant｜><think>\\n'}}{% endif %}"""
r1_tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE

In [66]:
r1_tokenizer.apply_chat_template(data['messages'], tokenize = False)

'<｜begin▁of▁sentence｜><｜User｜>2. In a computer game, a turtle moves across a grid on the computer screen, which contains 11 columns and 5 rows. Initially, it is located at the bottom-left corner of the screen - on the cell with coordinates $(0,0)$. If the program instructs the turtle to move off the screen, it reappears on the opposite side - for example, taking one step up from cell $(3,4)$, the turtle will end up in cell $(3,0)$. Where will the turtle be after executing the following program:\n1) 1 step down; 2) 2 steps to the right; 3) 3 steps up; 4) 4 steps to the left; 5) 5 steps down; 6) 6 steps to the right; .. ; 2016) 2016 steps to the left; 2017) 2017 steps down?<｜Assistant｜><think>\nOkay, so I need to figure out where the turtle ends up after executing a long series of movements on this 11x5 grid. The turtle starts at (0,0), which is the bottom-left corner. The movements are in a pattern: first 1 step down, then 2 steps right, 3 steps up, 4 steps left, 5 steps down, 6 steps r

In [76]:
df = pd.DataFrame(open_r1_ds['train']['messages'])

In [92]:
df[0] = df[0].apply(lambda x: x['content'])
df[1] = df[1].apply(lambda x: x['content'])

In [94]:
df['0_len'] = df[0].str.len()
df['1_len'] = df[1].str.len()

In [102]:
len(r1_tokenizer.encode(df[df['1_len'] == df['1_len'].max()][1].iloc[0]))

16509

In [103]:
len(r1_tokenizer.encode(df[df['0_len'] == df['0_len'].max()][0].iloc[0]))

2049

In [88]:
def token_len(series, tokenizer = r1_tokenizer):
    prompt = tokenizer.encode(series[0]['content'])
    response = tokenizer.encode(series[1]['content'])
    
    return len(prompt) + len(response)
    
df['len'] = df.apply(token_len, axis = 1)


KeyboardInterrupt



In [104]:
df[df['0_len'] == df['0_len'].max()][0].iloc[0]

'10. (10 points) There is a cup filled with a 16% saltwater solution. There are large, medium, and small iron balls, with volume ratios of 10: 4: 3. First, the small ball is submerged in the saltwater cup. As a result, 10% of the saltwater overflows, and the small ball is removed; then the medium ball is submerged in the saltwater cup and removed; next, the large ball is submerged in the saltwater cup and removed; finally, pure water is added to the cup until it is full. What is the concentration of the saltwater in the cup at this point? (保留一位小数) (保留一位小数 means "保留一位小数" in Chinese, which translates to "保留一位小数" in English. However, for clarity, I will replace it with "保留一位小数" in the final translation.)\n\n10. (10 points) There is a cup filled with a 16% saltwater solution. There are large, medium, and small iron balls, with volume ratios of 10: 4: 3. First, the small ball is submerged in the saltwater cup. As a result, 10% of the saltwater overflows, and the small ball is removed; then 